In [6]:

# system libraries
import os
import warnings

# ignoring all the warnings
warnings.filterwarnings('ignore')

# import data handling libraries

import numpy as np
import pandas as pd

# importing data visualisation libraries
import matplotlib.pyplot as plt
# matplotlib inline
import seaborn as sns
import sweetviz as sv
import scipy.stats







In [7]:
data= "."
os.listdir(data)

def calculate_accuracy(data,path_name,label,savepath):
        #Check if the lists have the same length
        new_data = data.sample(n=int(0.1*len(data)))
        out = new_data[new_data["RDlabel"]==new_data["UAlabel"]]
        #print(len(new_data),len(out))

        num_responders = len(out)
        num_tested =  len(new_data)    

        bayes_df = pd.DataFrame({'proportion': np.arange(0.0, 1.01, 0.01)})

        # compute the binomial likelihood of the observed data for each
        # possible value of proportion
        bayes_df['likelihood'] = scipy.stats.binom.pmf(num_responders,
                                                        num_tested,
                                                        bayes_df['proportion'])
        # The prior is equal for all possible values
        bayes_df['prior'] = 1 / bayes_df.shape[0]

        # compute the marginal likelihood by adding up the likelihood of each possible proportion times its prior probability.

        marginal_likelihood = (bayes_df['likelihood'] * bayes_df['prior']).sum()

        bayes_df['posterior'] = (bayes_df['likelihood'] * bayes_df['prior']) / marginal_likelihood

        # plot the likelihood, prior, and posterior
        plt.subplot(121)

        plt.title(f"count_{int(0.1*len(data))}")

        plt.plot(bayes_df['proportion'], bayes_df['likelihood'], label='likelihood')
        plt.plot(bayes_df['proportion'], bayes_df['prior'], label='prior')
        plt.plot(bayes_df['proportion'], bayes_df['posterior'],
                'k--', label='posterior')

        plt.legend()

        new_data = data.sample(n=len(data))
        out = new_data[new_data["RDlabel"]==new_data["UAlabel"]]
        num_responders =len(out)
        num_tested = len(new_data)

        # copy the posterior from the previous analysis and rename it as the prior

        study2_df = bayes_df[['proportion', 'posterior']].rename(columns={'posterior': 'prior'})

        # compute the binomial likelihood of the observed data for each
        # possible value of proportion

        study2_df['likelihood'] = scipy.stats.binom.pmf(num_responders,
                                                num_tested,
                                                study2_df['proportion'])

        # compute the marginal likelihood by adding up the likelihood of each possible proportion times its prior probability.

        marginal_likelihood = (study2_df['likelihood'] * study2_df['prior']).sum()

        study2_df['posterior'] = (study2_df['likelihood'] * study2_df['prior']) / marginal_likelihood

        # plot the likelihood, prior, and posterior
        plt.subplot(122)
        plt.title(f"count_{len(data)}")

        plt.plot(study2_df['proportion'], study2_df['likelihood'], label='likelihood')
        plt.plot(study2_df['proportion'], study2_df['prior'], label='prior')
        plt.plot(study2_df['proportion'], study2_df['posterior'],
                'k--', label='posterior')

        plt.legend()
        plt.savefig(f'{savepath}/{label}_user{path_name.split(".")[0]}.jpg')
        plt.close()

        mean=sum(study2_df['posterior']*study2_df['proportion'])*100
        print(mean)

        
        
        

In [8]:
# Load user annotations and reference data
# if we don't have header then we will use, pd.read_csv(r"D:\python\2018003737.csv", encoding="gbk", header=None)

path = r"D:\python\Zip file(1)\公众标注数据处理\input\student"
path_list = os.listdir(path)

for path_name in path_list:
    user_annotations = pd.read_csv(r"D:\python\Zip file(1)\公众标注数据处理\input\student\{}".format(path_name), encoding="gbk")
    reference_data = pd.read_csv(r"D:\python\41802.csv", encoding="gbk")

    user_annotations=user_annotations.add_prefix('UA')
    reference_data=reference_data.add_prefix('RD')

    def dataMapping(label):
        if label== "Elliptical galaxy": 
            return 0
        elif label=="Star":
            return 1
        elif label=="Clockwise-Z-wise spiral galaxy":
            return 2
        elif label=="Merger":
            return 3
        elif label=="Anti-clockwise-S-wise spiral galaxy":
            return 4
        elif label=="Don’t Know":
            return 5
        else:
            return 6
    user_annotations['UA_Num'] = user_annotations['UAlabel'].map(dataMapping)
    reference_data['RD_Num']=reference_data['RDlabel'].map(dataMapping)
    #print(reference_data)

    # I want to compare two files based on target data
    reference_data1= reference_data
    #print(user_annotations1)
    user_annotations1=user_annotations
    mrg=pd.merge(reference_data1, user_annotations1 , how='outer', left_on='RD41802', right_on=f'UA{path_name.split(".")[0]}')


    null=mrg.isnull().sum()
    null=mrg.dropna(how='any')
    # print(null)
    New1=null.to_csv(f'D:\\python\\user{path_name.split(".")[0]}.csv', index= False)
    #print(New)

    def compare_data_type(df):
        if df['RD_Num']==df['UA_Num']:
            return True
        else:
            return False
    
    null['data_compare']=null.apply(compare_data_type, axis=1)
    null[null['data_compare']==0]
    num=null['data_compare']
    #mrg1=pd.merge(null, num, how='outer', left_on='RD_Num', right_on='df' )
    #New1=null['data_compare'].to_csv('D:\python\mull1_n.csv', index= False)
    #print(num)
    #print(reference_data)
    reference_data=reference_data.set_index('RDlabel')
    user_annotations=user_annotations.set_index('UAlabel')
    df=null['RDlabel'].value_counts()
    df1=null['UAlabel'].value_counts()

    compare=pd.merge(df,df1,how='outer', left_index=True, right_index=True)

    reference_data_df=pd.DataFrame(reference_data)
    user_annotations=pd.DataFrame(user_annotations)

    import csv

    with open(f'D:\\python\\user{path_name.split(".")[0]}.csv', encoding='gbk') as csvfile : 
        reference_data= csv.DictReader(csvfile)
        count=0
        RD=[]

        for row in reference_data:
            count= count+1
        # print(float(row['RD_Num']))
            RD_1=RD.append(int(float(row["RD_Num"])))
            if count>4000:
                break
    labl=np.array(RD)
    data = pd.read_csv(f'D:\\python\\user{path_name.split(".")[0]}.csv')
    new_data = data.sample(n=len(data))

    out = new_data[new_data["RDlabel"]==new_data["UAlabel"]]
    #print(len(new_data),len(out))

    import csv

    with open(f'D:\\python\\user{path_name.split(".")[0]}.csv') as csvfile : 
        user_annotations= csv.DictReader(csvfile)
        count=0
        UA=[]

        for row in user_annotations:
            count= count+1
            # print(row['UA_Num'])
            UA_1=UA.append(int(float(row["UA_Num"])))
            if count>100:
                break
    predict=np.array(UA)

    savepath = "./Poior_result/"
    if not os.path.exists(savepath):
        os.makedirs(savepath)
 
    #-
    data = pd.read_csv(f'D:\\python\\user{path_name.split(".")[0]}.csv')
    label = "all"
    calculate_accuracy(data,path_name,label,savepath)
    class_galaxy = ["Elliptical galaxy","Star","Clockwise-Z-wise spiral galaxy","Merger"
                    ,"Anti-clockwise-S-wise spiral galaxy","Don’t Know","Else"]

    for class_name in class_galaxy:
        data = pd.read_csv(f'D:\\python\\user{path_name.split(".")[0]}.csv')
        data= data[data["RDlabel"]==class_name]
        label = class_name
        calculate_accuracy(data,path_name,label,savepath)

    
   
   

57.01316386540084
85.42976939239054
58.611111111111114
32.66666666666666
33.33333333333333
31.91489361702129
39.22651933701658
4.417559556654706
34.61393798038356
85.16226296721607
3.94947815336981
4.8163840857343185
4.878050662496369
7.082436231922999
4.935627317285932
14.626391116251018
46.92793806612695
94.35651882379761
1.8773629909824396
12.66666666666659
4.6911311817174015
13.51351351351348
13.398692810456831
1.0654963964315018
56.83159328350532
65.65217391304351
62.37623762376238
25.373134328358216
2.785813294523195
33.92857142857142
69.69696969696967
32.843137254901954
53.60544217891348
81.48535564853943
22.77777777777778
24.000000000000004
21.4285714283939
19.148936170212767
67.95580110497235
1.1780274779824706
55.495004542470625
81.4270724029414
20.555555555555568
32.66666666666666
23.80952380954078
48.936170212765944
74.03314917127072
1.3450446730738816
29.369042134071744
49.67391304347826
7.673267280370183
9.701492530797166
23.99999999999999
13.392857142884449
27.5482093663